**Download necessary files**

In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lakki\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

**Import the necessary libraries**

In [2]:
import numpy as np
import random
import string

**Import libraries for scraping - beautifulSoup**

In [3]:
import bs4 as bs
import urllib.request
import re

**Read the web link**

In [4]:
raw_html = urllib.request.urlopen('https://en.wikipedia.org/wiki/Natural_language_processing')
raw_html = raw_html.read()

**Convert the web link contents into a file**

In [5]:
article_html = bs.BeautifulSoup(raw_html, 'lxml')

**Find only the html text containing inside < p > tags**

In [6]:
article_paragraphs = article_html.find_all('p')

**Read all the text inside paragraphs**

In [7]:
article_text = ''

for para in article_paragraphs:
    article_text += para.text
    
article_text

'Natural language processing (NLP) is a subfield of linguistics, computer science, and artificial intelligence concerned with the interactions between computers and human language, in particular how to program computers to process and analyze large amounts of natural language data.  The goal is a computer capable of "understanding" the contents of documents, including the contextual nuances of the language within them. The technology can then accurately extract information and insights contained in the documents as well as categorize and organize the documents themselves.\nChallenges in natural language processing frequently involve speech recognition, natural language understanding, and natural language generation.\nNatural language processing has its roots in the 1950s. Already in 1950, Alan Turing published an article titled "Computing Machinery and Intelligence" which proposed what is now called the Turing test as a criterion of intelligence, a task that involves the automated inte

**Split the sentences**

In [8]:
corpus = nltk.sent_tokenize(article_text)
corpus

['Natural language processing (NLP) is a subfield of linguistics, computer science, and artificial intelligence concerned with the interactions between computers and human language, in particular how to program computers to process and analyze large amounts of natural language data.',
 'The goal is a computer capable of "understanding" the contents of documents, including the contextual nuances of the language within them.',
 'The technology can then accurately extract information and insights contained in the documents as well as categorize and organize the documents themselves.',
 'Challenges in natural language processing frequently involve speech recognition, natural language understanding, and natural language generation.',
 'Natural language processing has its roots in the 1950s.',
 'Already in 1950, Alan Turing published an article titled "Computing Machinery and Intelligence" which proposed what is now called the Turing test as a criterion of intelligence, a task that involves 

**Clean the text**

In [9]:
for i in range(len(corpus )):
    corpus [i] = corpus [i].lower()
    corpus [i] = re.sub(r'\W',' ',corpus [i])
    corpus [i] = re.sub(r'\s+',' ',corpus [i])

**Create a dictionary of word frequency**

In [11]:
wordfreq = {}
for sentence in corpus:
    tokens = nltk.word_tokenize(sentence)
    for token in tokens:
        if token not in wordfreq.keys():
            wordfreq[token] = 1
        else:
            wordfreq[token] += 1

**Filter the top 200 words**

In [12]:
import heapq
most_freq = heapq.nlargest(200, wordfreq, key=wordfreq.get)

**Compute IDF values**

In [13]:
word_idf_values = {}
for token in most_freq:
    doc_containing_word = 0
    for document in corpus:
        if token in nltk.word_tokenize(document):
            doc_containing_word += 1
    word_idf_values[token] = np.log(len(corpus)/(1 + doc_containing_word))

**Compute TF values**

In [14]:
word_tf_values = {}
for token in most_freq:
    sent_tf_vector = []
    for document in corpus:
        doc_freq = 0
        for word in nltk.word_tokenize(document):
            if token == word:
                  doc_freq += 1
        word_tf = doc_freq/len(nltk.word_tokenize(document))
        sent_tf_vector.append(word_tf)
    word_tf_values[token] = sent_tf_vector

**Compute TF-IDF values**

In [15]:
tfidf_values = []
for token in word_tf_values.keys():
    tfidf_sentences = []
    for tf_sentence in word_tf_values[token]:
        tf_idf_score = tf_sentence * word_idf_values[token]
        tfidf_sentences.append(tf_idf_score)
    tfidf_values.append(tfidf_sentences)

In [16]:
tf_idf_model = np.asarray(tfidf_values)

In [17]:
tf_idf_model = np.transpose(tf_idf_model)

In [18]:
tf_idf_model

array([[0.01725499, 0.00937076, 0.03249865, ..., 0.        , 0.        ,
        0.        ],
       [0.04806746, 0.06961139, 0.        , ..., 0.15231777, 0.15231777,
        0.15231777],
       [0.        , 0.04983542, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.02925846, 0.01588956, 0.02755321, ..., 0.        , 0.        ,
        0.        ],
       [0.01979248, 0.        , 0.03727787, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.06337238, ..., 0.        , 0.        ,
        0.        ]])

In [19]:
import pandas as pd
pd.DataFrame(tf_idf_model, columns=most_freq)

,of,the,to,and,in,language,a,natural,processing,nlp,...,program,amounts,goal,capable,contents,including,contextual,nuances,within,them
0,0.017255,0.009371,0.032499,0.061598,0.020533,0.058179,0.024292,0.051356,0.027144,0.030353,...,0.082017,0.082017,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.048067,0.069611,0.000000,0.000000,0.000000,0.036016,0.045113,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.152318,0.152318,0.152318,0.152318,0.152318,0.152318,0.152318,0.152318
2,0.000000,0.049835,0.000000,0.072798,0.036399,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.050049,0.050049,0.141811,0.000000,0.187772,0.066163,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.040607,0.000000,0.000000,0.088975,0.084036,0.000000,0.111272,0.117623,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.012941,0.021084,0.000000,0.030799,0.015400,0.014545,0.054657,0.019259,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.013195,0.021498,0.012426,0.015702,0.000000,0.014830,0.037152,0.019636,0.000000,0.046422,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.018693,0.020303,0.035207,0.000000,0.000000,0.042018,0.000000,0.055636,0.058811,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.014020,0.030455,0.000000,0.000000,0.066731,0.063027,0.039474,0.041727,0.088217,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.034807,0.037806,0.021853,0.027613,0.027613,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
